# Data Preparation for GeSAI - AB Data Challenge 

In [36]:
# Import necessary libraries
import pandas as pd

# 1. Preparación inicial del dataset AB3  

In [37]:
# Load dataset from data/data-ab3.parquet
df_ab3 = pd.read_parquet('../data/data_ab3.parquet')
print(df_ab3.head())

  POLIZA_SUMINISTRO NUMEROSERIECONTADOR  CONSUMO_REAL          FECHA_HORA  \
0  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           9.0 2024-01-01 00:29:14   
1  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 01:29:14   
2  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB          10.0 2024-01-01 02:29:14   
3  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 03:29:14   
4  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 04:29:14   

  DATA_INI_FACT DATA_FIN_FACT CREATED_MENSAJE CODIGO_MENSAJE TIPO_MENSAJE  
0    2024-01-24    2024-03-26             NaT           None         None  
1    2024-01-24    2024-03-26             NaT           None         None  
2    2024-01-24    2024-03-26             NaT           None         None  
3    2024-01-24    2024-03-26             NaT           None         None  
4    2024-01-24    2024-03-26             NaT           None         None  


## 1.1. Información básica de nuestro dataset

In [38]:
# Display basic information about the DataFrame
print('DataFrame Information:')
print(df_ab3.info())

# Print number of null values in each column
print('\nNumber of null values in each column:')
print(df_ab3.isnull().sum())

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121834 entries, 0 to 121833
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   POLIZA_SUMINISTRO    121834 non-null  object        
 1   NUMEROSERIECONTADOR  121834 non-null  object        
 2   CONSUMO_REAL         106983 non-null  float64       
 3   FECHA_HORA           121834 non-null  datetime64[us]
 4   DATA_INI_FACT        121834 non-null  object        
 5   DATA_FIN_FACT        121834 non-null  object        
 6   CREATED_MENSAJE      99400 non-null   datetime64[us]
 7   CODIGO_MENSAJE       99400 non-null   object        
 8   TIPO_MENSAJE         99400 non-null   object        
dtypes: datetime64[us](2), float64(1), object(6)
memory usage: 8.4+ MB
None

Number of null values in each column:
POLIZA_SUMINISTRO          0
NUMEROSERIECONTADOR        0
CONSUMO_REAL           14851
FECHA_HORA                 

## 1.2. Tratamiento de los valores nulos

### 1.2.1. Imputación en valores nulos de la variable "CONSUMO_REAL"
Dada la información proporcionada por el equipo de AB Data, cuando la variable "CONSUMO_REAL" tiene un valor NaN quiere decir que el valor registrado es un valor < 1, para el correcto funcionamiento de los modelos a desarrollar asumiremos que este valor NaN será "0".

In [ ]:
# Imputation of missing values in "CONSUMO_REAL" column
df_ab3.fillna({'CONSUMO_REAL': 0}, inplace=True)


Number of null values in CONSUMO_REAL after imputation: 0


### 1.2.2. Toma de decisiones sobre las variables "CREATED_MENSAJE", "CODIGO_MENSAJE" y "TIPO_MENSAJE"

Estas 3 variables hacen referencia a la detección y comunicación de fuga ('FUITA'), así como a su reiteración ('REITERACIÓ DE FUITA). Para llevar a cabo nuestro modelo predictivo nos hará falta simplificar el proceso de detección de fuga por lo que se ha decidido retirar estas columnas y añadir una nueva columna binaria que indica 0 si no hay fuga y 1 si hay fuga.

In [40]:
# Create binary column for leak detection
df_ab3['FUGA_DETECTADA'] = df_ab3['TIPO_MENSAJE'].apply(lambda x: 1 if x in ['FUITA', 'REITERACIÓ DE FUITA'] else 0)

# Drop unnecessary columns
df_ab3.drop(columns=['CREATED_MENSAJE', 'CODIGO_MENSAJE', 'TIPO_MENSAJE'], inplace=True)

### 1.2.3. Verificación de la imputación en valores nulos

Verificamos si hemos realizado una correcta imputación imprimiendo el número de valores nulos en cada columna.

In [44]:
# Print number of null values in each column
print('Number of null values in each column:')
print(df_ab3.isnull().sum())

Number of null values in each column:
POLIZA_SUMINISTRO      0
NUMEROSERIECONTADOR    0
CONSUMO_REAL           0
DATA_INI_FACT          0
DATA_FIN_FACT          0
FUGA_DETECTADA         0
FECHA                  0
HORA                   0
dtype: int64


## 1.3. Tratamiento de la variable "FECHA_HORA" 

Para una mayor claridad de nuestros datos se ha decidido separar la variable "FECHA_HORA" en dos variables distintas: "FECHA" y "HORA".


In [41]:
# Convert specified columns to datetime format
for col in ["FECHA_HORA"]:
    df_ab3[col] = pd.to_datetime(df_ab3[col], errors="coerce")

# Split 'FECHA_HORA' into separate date and time columns
df_ab3['FECHA'] = df_ab3['FECHA_HORA'].dt.date
df_ab3['HORA'] = df_ab3['FECHA_HORA'].dt.time

# Drop the original 'FECHA_HORA' column
df_ab3 = df_ab3.drop(columns=['FECHA_HORA'])

# Display updated DataFrame 
print('\nUpdated DataFrame:')
print(df_ab3.head())


Updated DataFrame:
  POLIZA_SUMINISTRO NUMEROSERIECONTADOR  CONSUMO_REAL DATA_INI_FACT  \
0  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           9.0    2024-01-24   
1  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0    2024-01-24   
2  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB          10.0    2024-01-24   
3  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0    2024-01-24   
4  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0    2024-01-24   

  DATA_FIN_FACT  FUGA_DETECTADA       FECHA      HORA  
0    2024-03-26               0  2024-01-01  00:29:14  
1    2024-03-26               0  2024-01-01  01:29:14  
2    2024-03-26               0  2024-01-01  02:29:14  
3    2024-03-26               0  2024-01-01  03:29:14  
4    2024-03-26               0  2024-01-01  04:29:14  
